<a href="https://colab.research.google.com/github/rohitnikam1/Basic-Financial-Analysis/blob/master/2_Variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measures of Dispersion

Dispersion measures how spread out a set of data is. This is especially important in finance because one of the main ways risk is measured is in how spread out returns have been historically. If returns have been very tight around a central value, then we have less reason to worry. If returns have been all over the place, that is risky.

Data with low dispersion is heavily clustered around the mean, while data with high dispersion indicates many very large and very small values.

Let's generate an array of random integers to work with.

In [1]:
import numpy as np
np.random.seed(121)

In [4]:
#Generate 20 random integers < 100 and sort them
x = np.random.randint(100, size=20)
x = np.sort(x)
print(f'x: {x}')
mu = np.mean(x)
print(f'mu: {mu}')

x: [ 3  4 10 13 14 21 30 36 41 41 45 51 55 68 76 82 83 87 94 94]
mu: 47.4


# Range

Range is simply the difference between the maximum and minimum values in a dataset. Not surprisingly, it is very sensitive to outliers. We'll use numpy's peak to peak (ptp) function for this.

In [3]:
print(f'Range of x: {np.ptp(x)}')

Range of x: 93


# Mean Absolute Deviation (MAD)

The mean absolute deviation is the average of the distances of observations from the arithmetic mean. We use the absolute value of the deviation, so that 5 above the mean and 5 below the mean both contribute 5, because otherwise the deviations always sum to 0.

$$ MAD = \frac{\sum_{i=1}^n |X_i - \mu|}{n} $$
where $n$ is the number of observations and $\mu$ is their mean.

In [10]:
abs_dispersion = np.mean([np.abs(i-mu) for i in x])
print(f'Mean absolute deviation of x: {abs_dispersion}')

Mean absolute deviation of x: 26.340000000000003


# Variance and Standard Deviation

The sample variance $\hat{\sigma^2}$ is defined as the average of the squared deviations around the mean:$$ \hat{\sigma^2} = \frac{\sum_{i=1}^n (X_i - \mu)^2}{n} $$

This is sometimes more convenient than the mean absolute deviation because absolute value is not differentiable, while squaring is smooth, and some optimization algorithms rely on differentiability.

Standard deviation is defined as the square root of the variance, $\hat{\sigma}$, and it is the easier of the two to interpret because it is in the same units as the observations.

In [12]:
var = np.var(x)
print(f'Variance of x: {var}')
std = np.std(x)
print(f'Standard deviation of x: {std}')

Variance of x: 922.9399999999999
Standard deviation of x: 30.379927583850492


One way to interpret standard deviation is by referring to *Chebyshev's inequality*. This tells us that the proportion of samples within $k$ standard deviations (that is, within a distance of $k \cdot$ standard deviation) of the mean is at least $1 - 1/k^2$ for all $k>1$.

Let's check that this is true for our data set.

In [27]:
k = 1.25
dist = k*std
x_within_dist = [i for i in x if np.abs(i-mu) < dist]
print(f'Observation within {k} standard deviations: {x_within_dist}')
print(f'Confirming that {len(x_within_dist)/len(x)} > {1-k**-2}')

Observation within 1.25 standard deviations: [10, 13, 14, 21, 30, 36, 41, 41, 45, 51, 55, 68, 76, 82, 83]
Confirming that 0.75 > 0.36


The bound given by Chebyshev's inequality seems fairly loose in this case. This bound is rarely strict, but it is useful because it holds for all data sets and distributions.

# Semivariance and Semideviation

Although variance and standard deviation tell us how volatile a quantity is, they do not differentiate between deviations upward and deviations downward. Often, such as in the case of returns on an asset, we are more worried about deviations downward. This is addressed by semivariance and semideviation, which only count the observations that fall below the mean. Semivariance is defined as
\begin{align}
\frac{\sum_{X_i \lt \mu} (X_i - \mu)^2}{n_{\lt}}
\end{align}
where $n_{\lt}$ is the number of observations which are smaller than the mean. Semideviation is the square root of the semivariance.

In [30]:
# Since there is no in-built function for semivariance, we calculate it here.
lows = [i for i in x if i < mu]
semivar = np.sum((np.array(lows)-mu)**2)/len(lows)
print(f'Semivariance of x: {semivar}')
print(f'Semideviation of x: {np.sqrt(semivar)}')

Semivariance of x: 797.2690909090907
Semideviation of x: 28.235953869297397


A related notion is the target semivariance (and target semideviation), where we average the distance from a target of values which fall below that target: 
\begin{align}
 \frac{\sum_{X_i \lt B} (X_i - B)^2}{n_{\lt B}}
 \end{align}

In [32]:
B = 19
lows_B = [i for i in x if i < B]
semivar_B = np.sum((np.array(lows_B)-B)**2)/len(lows_B)
print(f'Semivariance of x for target {B}: {semivar_B}')
print(f'Semideviation of x for target {B}: {np.sqrt(semivar_B)}')

Semivariance of x for target 19: 124.6
Semideviation of x for target 19: 11.16243700990066


# These are Only Estimates

All of these computations will give you sample statistics, that is standard deviation of a sample of data. Whether or not this reflects the current true population standard deviation is not always obvious, and more effort has to be put into determining that. This is especially problematic in finance because all data are time series and the mean and variance may change over time. In general do not assume that because something is true of your sample, it will remain true going forward.

## References

- "Quantitative Investment Analysis", by DeFusco, McLeavey, Pinto, and Runkle